In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

In [ ]:
filepath = "iot23_original.csv"
df_iot = pd.read_csv(filepath)
del df_iot['Unnamed: 0']

In [ ]:
df_iot= df_iot.drop(columns=['Timestamp', 'Src_IP','Dst_IP', 'Dst_Port','Src_Port','uid','service','conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history'])
#필요없는 피처 drop

In [ ]:
#df_iot.to_csv('IoT_23_process.csv')

In [ ]:
filepath1 = "IoT_23_process.csv"
df_iot23 = pd.read_csv(filepath1)
del df_iot23['Unnamed: 0']

In [ ]:
filepath1 = "all_data.csv"
df_iot20 = pd.read_csv(filepath1)

In [ ]:
df_iot23= df_iot23.reindex(columns = df_iot23.columns.tolist() + ['Flow_Pkts/s', 'Flow_IAT_Mean', 'Flow_IAT_Max', 'Bwd_IAT_Tot',  'Bwd_IAT_Max', 'Init_Bwd_Win_Byts'])
#공통 피처 9개에 상위 6개 피처 추가

In [ ]:
df_iot20= df_iot20.drop(columns=['Bwd_IAT_Mean','Src_Port', 'Src_IP','Dst_IP', 'Dst_Port','Timestamp','Flow_ID','TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Min','Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min','Bwd_Pkt_Len_Std', 'Flow_Byts/s',  'Flow_IAT_Std', 'Flow_IAT_Min', 'Fwd_IAT_Tot', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',  'Bwd_IAT_Std', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean', 'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt', 'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg', 'Fwd_Seg_Size_Avg', 'Bwd_Seg_Size_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg', 'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Subflow_Fwd_Pkts', 'Subflow_Fwd_Byts', 'Subflow_Bwd_Pkts', 'Subflow_Bwd_Byts', 'Init_Fwd_Win_Byts', 'Fwd_Act_Data_Pkts', 'Fwd_Seg_Size_Min', 'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean', 'Idle_Std', 'Idle_Max', 'Idle_Min', 'Label','Sub_Cat'])
#필요없는 피처 drop

In [ ]:
df_iot20 = df_iot20.rename(columns={'Protocol':'proto','Flow_Duration':'duration','Fwd_Pkt_Len_Mean':'orig_bytes','Bwd_Pkt_Len_Mean':'resp_bytes','Tot_Fwd_Pkts':'orig_pkts', 'Fwd_Pkts/s':'orig_ip_bytes', 'Tot_Bwd_Pkts':'resp_pkts', 'Bwd_Pkts/s':'resp_ip_bytes', 'Cat': 'label'})

In [ ]:
#df_iot20.to_csv('IoT_20_process.csv')

In [ ]:
frames=[df_iot23,df_iot20]

In [ ]:
df_combined =pd.concat(frames)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
X = df_combined[['proto', 'duration', 'orig_bytes', 'resp_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'Flow_Pkts/s', 'Flow_IAT_Mean', 'Flow_IAT_Max', 'Bwd_IAT_Tot', 'Bwd_IAT_Max', 'Init_Bwd_Win_Byts']]
Y = df_combined['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
Z = df_iot23[['proto', 'duration', 'orig_pkts', 'resp_pkts', 'orig_bytes', 'resp_bytes', 'Flow_Pkts/s', 'Flow_IAT_Mean', 'Flow_IAT_Max', 'Bwd_IAT_Tot', 'Bwd_IAT_Max', 'orig_ip_bytes', 'resp_ip_bytes', 'Init_Bwd_Win_Byts']]
W = df_iot23['label']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
X_test = Z_test
Y_test = W_test

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()

In [ ]:
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print('\n')

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))